In [ ]:
import sys
sys.path.append("../lib")
from models import build_ao_model
from generators import AudioGenerator
import scipy.io.wavfile as wavfile
import utils
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
MIX_TRAIN = "../data/audio_train/mix"
MIX_TEST = "../data/audio_test/mix"
CRM_TRAIN = "../data/audio_train/crm"
CRM_TEST = "../data/audio_test/crm"

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", 
                           min_delta=0, 
                           patience=3, 
                           verbose=True, 
                           mode="auto", 
                           baseline=None, 
                           restore_best_weights=False)
callbacks = [early_stop]

In [ ]:
epochs = 100
initial_epoch=0
batch_size = 6
n_speakers = 2
ao_model = build_ao_model(n_speakers)

In [ ]:
mix_train_files = utils.get_files(MIX_TRAIN)
crm_train_files = utils.get_files(CRM_TRAIN)

mix_test_files = utils.get_files(MIX_TEST)
crm_test_files = utils.get_files(CRM_TEST)

In [ ]:
train_generator = AudioGenerator(mix_train_files, crm_train_files, n_speakers, batch_size)

In [ ]:
ao_model.fit_generator(generator=train_generator, 
                       epochs=epochs,
                       initial_epoch=initial_epoch)

In [ ]:
mix = np.load(mix_test_files[5])
cRMs = ao_model.predict(np.expand_dims(mix, axis=0))
cRMs = cRMs[0]
for i in range(2):
    cRM = cRMs[:,:,:,i]
    assert cRM.shape == (298,257,2)
    F = utils.icRM(mix,cRM)
    T = utils.istft(F)
    filename = str(i) + '.wav'
    wavfile.write(filename,16000,T)